In [87]:
## Import the Requests package and call the API with the parameters of your choice.
import requests

# Parameter of a choice - page. Geting all jobs..
jobs = []
found_all = False
page = 0
while not found_all:
    payload = {'page': page}
    r = requests.get('https://jobs.github.com/positions.json', params=payload)
    #print(r.url)
    ## Print the status code to make sure it went well.
    #print(r.status_code)
    r_json = r.json()
    if len(r_json) == 0:
        found_all = True
    else:
        jobs = jobs + r_json
    page += 1
print(len(jobs))

122


In [88]:
## Import json and print the whole json response with the json.dumps() function. See what kind of data it contains.
import json
#printing first 1000 characters of a result of json dump, which is a string
print(json.dumps(jobs, sort_keys=True, indent=2)[:1000])


## Get the dictionary of the first job and call the method keys() to have a better idea of the data at hand.

## I'm getting all keys, because I'm not sure if there is any optional data, and I don't know how it's represented
job_map = {}
for job in jobs:
    for key in job.keys():
        if key not in job_map:
            job_map[key] = 0
        job_map[key] += 1
print(job_map)

## Print some of the first job's content, including images and HTML. You might need to import the IPython package.
from IPython.core.display import display, HTML, Image
display(Image(url=jobs[0]['company_logo']))
display(HTML(jobs[0]['description']))


[
  {
    "company": "GitHub",
    "company_logo": "http://github-jobs.s3.amazonaws.com/c469c478-0296-11e6-87ae-ffc3f1270dff.png",
    "company_url": "https://github.com/about/jobs",
    "created_at": "Fri Dec 01 20:49:16 UTC 2017",
    "description": "<p>GitHub is looking for a Senior Quality Engineer to help grow our testing efforts by contributing towards a growing number of product offerings. This person will work closely with Product, Development and Design at all stages of the product lifecycle as an advocate for standard methodologies that aid in delivering high quality products while being cognizant of the importance of time-to-market.</p>\n\n<p>We\u00a0value collaboration,\u00a0empathy,\u00a0quality,\u00a0positive\u00a0impact and shipping. You will excel when your work reflects these values.</p>\n\n<h3>Responsibilities:</h3>\n\n<ul>\n<li>Collaborate across teams to build a comprehensive test strategy with an automated approach for new features as well as to maintain existing r

In [51]:
## Import SQLite and create a database.
import sqlite3
sqlite_file = 'github.sqlite'

## Create 4 tables: company, job, location, github_metadata. 
## Pay sppecial attention on primary and foreign keys as well as unique and non-nullable columns.

tables = [
    'create table company ( \
        id integer primary key, \
        company text unique not null, \
        url text, \
        logo_url text)', 
    'create table location ( \
        id integer primary key, \
        location text unique not null)', 
    'create table job ( \
        id integer primary key, \
        company_id integer not null, \
        location_id integer not null, \
        job_type text, \
        title text, \
        description text, \
        how_to_apply text, \
        created_at datetime , \
        created_at_raw text , \
        foreign key(company_id) references company(id), \
        foreign key(location_id) references location(id))', 
    'create table github_metadata ( \
        github_id text primary key, \
        url text not null, \
        job_id integer not null, \
        foreign key(job_id) references job(id) \
        )']
## Make sure you store 'created_at' as a datetime and not a string.

conn = sqlite3.connect(sqlite_file)
c = conn.cursor()
c.execute('drop table if exists github_metadata')
c.execute('drop table if exists job')
c.execute('drop table if exists company')
c.execute('drop table if exists location')

for stmt in tables:
    c.execute(stmt)
conn.commit()

c.execute('SELECT type, count(*) FROM sqlite_master group by type')
all_rows = c.fetchall()
print(all_rows)
conn.close()


[('index', 3), ('table', 4)]


In [ ]:
## Store the data you got from the API request in the database.
## Make sure you create a transaction (i.e., if an error occurs while saving a job's location data
## the database should rollback on all statements regarding that job).

import sqlite3
from datetime import datetime

In [103]:
# Prep...

def get_company_id(conn, company, logo, url):
    c = conn.cursor()
    c_id = None
    try:
        c.execute("INSERT INTO company(company, logo_url, url) VALUES (?,?,?)", (company, logo, url))
        conn.commit()
    except sqlite3.IntegrityError as e:
        pass
    except sqlite3.Error as e:
        conn.rollback()
        print('Can\'t insert company...')
        print(e)

    c.execute("select id from company where company=?", [company])
    c_id = c.fetchone()[0]
    
    return c_id

def get_location_id(conn, location):
    c = conn.cursor()
    loc_id = None
    try:
        c.execute("INSERT INTO location(location) VALUES (?)", ([location]))
        conn.commit()
    except sqlite3.IntegrityError as e:
        pass
    except sqlite3.Error as e:
        conn.rollback()
        print('Can\'t insert location...')
        print(e)

    c.execute("select id from location where location=?",  [location])
    loc_id = c.fetchone()[0]
    
    return loc_id

def get_job_id(conn, company_id, loc_id, description, job_type, title, how_to_apply, created_at, created_at_raw):
    c = conn.cursor()
    job_id = None
    try:
        c.execute("INSERT INTO \
        job(company_id, location_id, description, job_type, title, how_to_apply, created_at, created_at_raw) \
        VALUES (?,?,?,?,?,?,?,?)", (company_id, loc_id, description, job_type, title, how_to_apply, created_at, created_at_raw))
        conn.commit()
    except sqlite3.IntegrityError as e:
        pass
    except sqlite3.Error as e:
        conn.rollback()
        print('Can\'t insert job...')
        print(e)

    c.execute("select id from job where rowid=last_insert_rowid()")
    job_id = c.fetchone()[0]

    return job_id

def process_job(conn, info):
   
    c = conn.cursor()
    company = info['company']
    company_url = info['company_url']
    #created_at = datetime.strptime(info['created_at'], '%a %b %d %H:%M:%S %Z %Y').strftime('%Y-%m-%d %H:%M:%S.000')
    created_at = datetime.strptime(info['created_at'], '%a %b %d %H:%M:%S %Z %Y')
    created_at_raw = info['created_at']
    logo = info['company_logo']
    how_to_apply = info['how_to_apply']
    description = info['description']
    location = info['location']
    job_type = info['type']
    title = info['title']
    gh_id = info['id']
    url = info['url']
    
    c.execute("select github_id from github_metadata where github_id=?", [gh_id])
    gh_id_exist = c.fetchone()
    
    if gh_id_exist is None:
        loc_id = get_location_id(conn, location)
        company_id = get_company_id(conn, company, logo, company_url)
        job_id = get_job_id(conn, company_id, loc_id, description, job_type, title, how_to_apply, created_at, created_at_raw)
        
        try:
            c.execute("INSERT INTO \
            github_metadata(github_id, url, job_id) \
            VALUES (?,?,?)", (gh_id, url, job_id))
            conn.commit()
        except sqlite3.IntegrityError as e:
            pass
        except sqlite3.Error as e:
            conn.rollback()
            print('Can\'t insert github metadata...')
            print(e)


In [ ]:
# YZ - QUESTION:
# In GET_COMPANY_ID parameters that I pass to INSERT c.execite with tuple (company, logo, url) 
# In GET_LOCATION_ID parameters that I pass to INSERT c.execite with list [location] because 
# somehow c.execute treats each character of this 'location' variable as an input 
# as opposed to the whole string. Why does it do it? 
# Also, same for SELECT c.execute calls in other methods: when I tried to pass tuple, i.e "(column)" it complained, 
# when I changed it to list, i.e."[column]", it worked
# This behaviour seems inconsistent to my. 

In [ ]:
# Actual processing

sqlite_file = 'github.sqlite'
conn = sqlite3.connect(sqlite_file)
c = conn.cursor()

for entry in jobs:
    try:
        process_job(conn, entry)
    except sqlite3.Error as e:
        print(e)
        conn.rollback()
conn.close()
print('done...')


In [114]:
## Execute simple SQL queries in each table and make sure everything looks as was intended.

sqlite_file = 'github.sqlite'
conn = sqlite3.connect(sqlite_file)

c = conn.cursor()
c.execute(
    "select 'company' as tbl, count(*) as row_count from company union all \
    select 'location' as tbl, count(*) as row_count from location union all \
    select 'jo' as tbl, count(*) as row_count from job union all \
    select 'github_metadata' as tbl, count(*) as row_count from github_metadata")
all_rows = c.fetchall()
print('Table Counts: %s' % (all_rows))


Table Counts: [('company', 82), ('location', 83), ('jo', 122), ('github_metadata', 122)]


In [115]:
## Create queries the following queries:
## Find out how many jobs have the phrase "Data Science" and how many have the word "Python" in the description
c.execute(
    "select count(*) from job \
    where lower(description) like '%data science%' and lower(description) like '%python%'")
all_rows = c.fetchone()[0]
print('\nData Science && Python jobs: %i' % (all_rows))



Data Science && Python jobs: 7


In [116]:
c.execute(
    'select count(*) \
    from github_metadata m, job j, company c, location l \
    where m.job_id=j.id and j.company_id=c.id and j.location_id=l.id ')
all_rows = c.fetchone()[0]
print('\nJoin Count: %i' %(all_rows))\




Join Count: 122


In [117]:
c.execute(
    "select 'company' as tbl, 'null company url' as issue, count(*) from company where url is null union all \
    select 'company' as tbl, 'null logo' as issue, count(*) from company where logo_url is null union all \
    select 'job' as tbl, 'distinct job types' as issue, count(distinct job_type) from job union all \
    select 'job' as tbl, 'future created_at' as issue, count(*) from job where date(created_at) > date('now') union all \
    select 'job' as tbl, 'null how_to_apply' as issue, count(*) from job where how_to_apply is null union all\
    select 'location' as tbl, 'duplicate locations' as issue, count(*) from \
        (select i.location, o.location \
        from location i, location o \
        where i.id > o.id \
            and trim(lower(i.location))=trim(lower(o.location))\
        )"
    )
all_rows = c.fetchall()
print('\nData Issues:')
print(all_rows)


Data Issues:
[('company', 'null company url', 16), ('company', 'null logo', 14), ('job', 'distinct job types', 2), ('job', 'future created_at', 5), ('job', 'null how_to_apply', 0), ('location', 'duplicate locations', 7)]


In [118]:
c.execute('select i.location, o.location from location i, location o\
        where i.id > o.id \
            and trim(lower(i.location))=trim(lower(o.location))')

all_rows = c.fetchall()
print('\nData Issues Details:')
print(all_rows)


Data Issues Details:
[('United States', 'United States '), ('London, UK ', 'London, UK'), ('remote', 'Remote'), ('San Francisco ', 'San Francisco'), ('Remote ', 'Remote'), ('Remote ', 'remote'), ('San Francisco or Remote ', 'San Francisco or Remote')]


In [119]:
## Find out how many were created in the past 7 days
c.execute(
    "select count(*) from job \
    where date(created_at) > date('now', '-7 days') and date(created_at) < date('now')")
all_rows = c.fetchone()[0]
print('\nJobs created in the past 7 days: %i' % (all_rows))



Jobs created in the past 7 days: 27


In [120]:
## Find out how are "Remote Ok" and which companies those are with
c.execute(
    "select \
        company, \
        count(*) as num_oj_jobs \
    from company c, job j, location l \
    where \
        j.company_id=c.id and j.location_id=l.id and \
        ((lower(j.description) like '%remote%') or (lower(l.location) like '%remote%'))\
    group by company")
all_rows = c.fetchall()
print('\nRemote-friendly companies with # of remote jobs:')
print(all_rows)

conn.close()


Remote-friendly companies with # of remote jobs:
[('Aeolus Robotics, Inc.', 1), ('Ben Kinney Companies Tech Division', 2), ('Cavulus', 2), ('Citrusbyte', 1), ('FlexJobs', 1), ('FullStack', 1), ('GitHub', 6), ('Hotjar', 1), ('JOOR, Inc.', 1), ('Kaggle', 1), ('Kissinger Associates, Inc.', 1), ('Paktor Pte Ltd', 1), ('Rails Machine', 1), ('ustwo London Ltd', 1)]


In [121]:
print('\n\n\nDONE...')




DONE...
